<a href="https://colab.research.google.com/github/sainivas-99/License-Plate-Detection/blob/main/LPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics -qq
!pip install easyocr -qq
!pip install GPUtil -qq
!pip install pytesseract -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import os
import shutil
import yaml
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

import IPython.display as display
import cv2

import ultralytics
from ultralytics import YOLO
import IPython.display as display
import easyocr
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import pytesseract
import xml.etree.ElementTree as xt
from bs4 import BeautifulSoup
import glob

In [6]:
def get_bbox_coordinates(path):
    parsed = xt.parse(path)
    root = parsed.getroot()
    img_object = root.find('object')

    labels_info = img_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    return  xmin, xmax, ymin, ymax

def draw_boundingbox(coordinates, img, title = '' ,thickness=1):
    coordinate_values = coordinates[['xmin','xmax','ymin','ymax']].values.astype(int)
    classes = coordinates['class'].values

    for i,detection in enumerate(coordinate_values):
        begin = (detection[0], detection[-1])
        end = (detection[1], detection[2])
        detected_class = classes[i]
        if detected_class == 'car_plate':
            number_plate = coordinates['plate_number'].values[i]
            cv2.rectangle(img, begin, end, (0,0,190), thickness)
            cv2.putText(img=img, text=f'{detected_class} ',
                org= (detection[0], detection[2]-20),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 0, 255),thickness=2)
            cv2.putText(img=img, text=f'{number_plate}',
                org= (detection[0]-10, detection[-1]+30),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 0, 255),thickness=2)
        else:
            cv2.rectangle(img, begin, end, (255,0,0), thickness)

            cv2.putText(img=img, text=f'{detected_class}',
                org= (detection[0], detection[2]-20),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=2)

    return img


def make_folder(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [7]:
image_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ML/datasets/images/*.png')
xml_annotations_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ML/datasets/annotations/*.xml')

annotations_tree = xt.parse(xml_annotations_path[0])
xt.dump(annotations_tree)

<annotation>
    <folder>images</folder>
    <filename>Cars0.png</filename>
    <size>
        <width>500</width>
        <height>268</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>licence</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>226</xmin>
            <ymin>125</ymin>
            <xmax>419</xmax>
            <ymax>173</ymax>
        </bndbox>
    </object>
</annotation>


In [8]:
index = np.arange(len(xml_annotations_path))
np.random.shuffle(index)

val_indices = index[:50]
train_indices = index[50:]

print('Train Size: ', len(train_indices))
print('Valid Size: ', len(val_indices))

Train Size:  383
Valid Size:  50
